# Evaluation Metrics

In [ ]:
# import modules here


## Import the data
We're using the flights dataset from before

In [ ]:
data = pd.read_csv('data/flights08_clean.csv')

In [ ]:
data.head()

## Classification metrics

Run the code below to perform a train test split, and fit a 5-nearest neighbor classifier.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('MajorDelay', axis=1), 
                                                    data['MajorDelay'], 
                                                    test_size=0.3, 
                                                    random_state=1337,
                                                    stratify=data['MajorDelay'])

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

Get y_hat - the classification predictions

In [ ]:
y_hat = ...


Calculate the accuracy of the predictions

In [ ]:
accuracy = (y_hat==y_test).mean()
accuracy

Why is this a bad metric for this data? Calculate the accuracy for each class. Then get the mean class accuracy.

Calculate the confusion matrix using sklearn.metrics.confusion_matrix. Use the given code to plot it, but make sure you label the rows and columns correctly...

In [ ]:
cm = ...
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel(...)
plt.ylabel(...)


Get the number of:
* true positives
* false positives
* true negatives
* and false negatives

and arrange them in a matrix to match the confusion matrix above

In [ ]:
nr_true_positives = ...
nr_false_positives = ...
nr_true_negatives = ...
nr_false_negatives = ...


manual_cm = np.array([
    [..., ...],
    [..., ...]
])

sns.heatmap(manual_cm, annot=True, fmt='d')

Get the precision, recall, and f1 score

Check the values against sklearn.metrics.classification_report

## Assessing probability outputs

### ROC curves

The metric "Area Under the ROC Curve", or AUC, doesn't care about how 'accurate' the probability is, only the ordering of the probabilities. If, when you order your data by the predicted probability and look at their true classes, a perfect ordering is when the classes are split perfectly. For example:

This data is perfectly ordered
```
class   0,   0,   1,   1
prob   .1,  .1,  .9, .99
```

As is this --
```
class   0,   0,   1,   1
prob  .01,  .1, .11, .99
```

Whilst the first has 'better' probabilities, for both of these classifiers (that produced the data above), there exists a threshold which you could use to get perfect classification. They will both have the same ROC curve.

The ROC curve plots the true positive rate (yet another name for recall or sensitivity i.e. nr_true_positives / nr_positive_cases) against the false_positive_rate (nr_false_positives / nr_negative_cases). A line is drawn by plotting the chages as the threshold moves from 1 (everything predicted negative - bottom left corner) to 0 (everything predicted positive - top right corner).

Here's an example from wikipedia:

![img](https://upload.wikimedia.org/wikipedia/commons/6/6b/Roccurves.png)

#### Write a function to get the points on an ROC curve
The function should take as input:
* The true labels y
* The predicted probabilities y_pred

and outputs:
* fpr_array - a numpy array of false positive rates
* tpr_array - the corresponding array of true positive rates
* thresholds - the corresponding array of thresholds that obtained that tpr, fpr pair

This is quite tricky, so we provide hints in the function skeleton below.

In [ ]:
def get_fpr_tpr(y, y_pred, thresh):
    # Write a function which gets the false positive rate
    # and true positive rate when given a threshold
    tpr = ...
    fpr = ...
    return fpr, tpr


def my_roc_curve(y, y_pred):
    # we need only check a finite number of thresholds
    # get the unique values in y_pred and sort them
    thresh = ...
    
    
    # Loop through all the thresholds and record the 
    # fpr and tpr in an array
    return fpr_array, tpr_array, thresh

In [ ]:
# we can now test this for our knn and plot
y_pred = knn.predict_proba(X_test)[:,1]
fpr, tpr, thresh = my_roc_curve(y_test, y_pred)
plt.plot(np.append(fpr, 0), np.append(tpr, 0), 'o-')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

In [ ]:
# And we can check it against the sklearn implementation
from sklearn.metrics import roc_curve
fpr, tpr, thresh = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr, 'o-')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.show()

#### Get the AUC with sklearn.metrics.roc_auc_score
The AUC is simply the area under this curve, the larger the better. The best possible scenario is that you have an AUC of 1 which means there is a point in the top left corner i.e. tpr=1 and fpr=0.

#### Extension: Fit a Logistic Regression and a Decision Tree Classifier and compare their ROC curves and AUC
A couple of things to think about:
* Which classifier is best and why?
* Why does knn only have a few points whereas logistic has many?
* Why does a decision tree to max depth only have one point?

Where to next? Try the [brier score](http://scikit-learn.org/stable/modules/model_evaluation.html#brier-score-loss) - This aims to measure classification probability quality, not simply ordering. 

For regression, see the [r2_score](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination).